# Laboratorio de Métodos Numéricos 
## Matrices SDP y factorización de Cholesky

En este taller, vamos a implementar la factorización de Cholesky.

Primero, algún código preliminar para correr algunos tests

In [2]:
pip install numpy

     |████████████████████████████████| 17.3 MB 30.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import math 
import traceback
eps = 1e-5

def isSymmetrical(A):
    return np.allclose(A,A.T,eps)

def validateMe(L,A):
    return np.allclose(L@L.T,A)

def mntest(func):
    global tests
    
    tests.append(func)    
    
    return func

def correr_tests():
    excepciones = []
    for test in tests:
        try:
            print("Corriendo {} ... ".format(test.__name__), end='')
            test()
            print("OK")
        except AssertionError as e:
            error_msg = traceback.format_exc()
            excepciones.append((test, error_msg))
            print("ERROR")
    
    if len(excepciones) > 0:
        print("\nErrores:\n")
        for (test, error_msg) in excepciones:
            print("En {}".format(test.__name__))
            print(error_msg)
    else:
        print("Todos los tests pasaron correctamente")



### Ejercicio 1: 
Completar la función que calcula la factorización de cholesky.
Recibe una matriz A y debe devolver una nueva matriz con la L de cholesky.
Debe chequear que la matriz sea cuadrada, simétrica, y luego durante el algoritmo agregar los chequeos necesarios donde haga falta.
En cualquier caso que falle, debe lanzar una excepción con el mensaje "Matriz no SDP."

In [107]:
def cholesky(A):
    
    if(not isSymmetrical(A)):
        raise ValueError("Matriz no SDP")
        
    n = A.shape[0];
    L = np.zeros((A.shape[0],A.shape[1]), float)
    
    for i in range(0,n):
       
        """
        TODO: Completar código (1-3 líneas aprox)
        """
        T = A[i][i] - np.sum(np.array([L[i][k]**2 for k in range(0,i)]))
        if T < 0:
            raise ValueError("Sorry, no numbers below zero")
            
        L[i,i] = np.sqrt(T)   
       
        
        for j in range(i+1,n):
            
            """
            TODO: Completar (1-3 líneas aprox)
            """
            if L[i,i] == 0:
                raise ValueError("Divisor cant be zero")
            S = np.array([L[j][k]*L[i][k] for k in range(0,i)])
            L[j,i] = (1/L[i,i])*(A[j,i] - np.sum(S))
        
    return L



In [108]:
tests = []

@mntest
def matriz_no_simetrica():
    A = np.array([
          [0.066076,.181880],
          [1.181880,.624953]
    ])

    try:
        L = cholesky(A)
        raise AssertionError("Tiene que lanzar excepción")
    except ValueError as e:
        pass


@mntest
def matriz_no_sdp():
    A = np.array([
          [-0.066076,   0.181880],
          [ 0.181880,   0.624953]
    ])
    try:
        L = cholesky(A)
        raise AssertionError("Tiene que lanzar excepción")
    except ValueError as e:
        pass

    
@mntest
def matriz_diagonal_positiva():
    A = np.array([
          [4.0,.0],
          [.0,9.0]
    ])
    L_1 = np.array([
          [2.0,0.0],
          [.0,3.0]
    ])
    
    L2 = cholesky(A)
    
    assert(np.allclose(L2,L_1))
    
    
@mntest
def matriz_sdp1():
    A  = np.array([[4.,-1.,1.], [-1, 4.25, 2.75],[1,2.75,3.5]],dtype='f')
    L = cholesky(A)
    
    assert(np.allclose(A,L@L.T))
    
    
@mntest
def matriz_sdp2():
       
    L_1 =  np.array([
        [5.99246,   .0,  .0,   .0,   .0],
        [0.45174,   5.02065,   .0,   .0,   .0],
        [0.50921,   0.40611,   5.25767,   .0,   .0],
        [0.45632,   0.52172,   0.78430,   5.98219,   .0],
        [0.41578,   0.11483,   0.24938,   0.45188,   5.28701]
    ])

    A    = L_1@L_1.T
    L2   = cholesky(A)
    assert(np.allclose(L2,L_1))

    
correr_tests()


Corriendo matriz_no_simetrica ... OK
Corriendo matriz_no_sdp ... OK
Corriendo matriz_diagonal_positiva ... OK
Corriendo matriz_sdp1 ... OK
Corriendo matriz_sdp2 ... OK
Todos los tests pasaron correctamente


### Ejercicio 2: 
Completar la función que calcula la factorización de cholesky por bloques.
Recibe una matriz A y debe devolver una nueva matriz con la L de cholesky.
Tener en cuenta las mismas consideraciones que el ejercicio anterior.


### ¿Cómo es esto de resolver en bloques?
Sea $A \in \mathbb{R}^{n \times n}$ a la cual le queremos calcular la factorización Cholesky. Para ello, lo calcularemos en bloques en vez del método antes visto

Supongamos que A tiene factorización Cholesky; es decir podemos descomponerla en $L y L^T \in \mathbb{R}^{n \times n}$ escribiendo la descomposición en bloques de la forma:


$$
A = L L^T \\
$$

$$
\begin{pmatrix}
a_{11}   & A_{12}  \\
A_{12}^T & A_{22}
\end{pmatrix} = \begin{pmatrix}
l_{11} & 0  \\
L_{21} & L_{22}
\end{pmatrix} \begin{pmatrix}
l_{11} & L_{21}^T  \\
0      & L_{22}
\end{pmatrix}
$$

&nbsp;


donde $a_{11}, l_{11}$ son escalares, $A_{12} \in \mathbb{R}^{1 \times (n-1)}$ (vector fila), $L_{21} \in \mathbb{R}^{(n-1) \times 1}$ (vector columna)

De aquí obtenemos las siguientes ecuaciones:

$$
a_{11} = l_{11}^2  \\
A_{12} = L_{21}^T l_{11} \\
A_{22} = L_{21} L_{21}^T + L_{22}L_{22} \\
$$

Estas 3 ecuaciones nos permitirán despejar y obtener el contenido del primer escalar y de la columna de L. Ahora, nos queda la última ecuación:

$$
A_{22} = L_{21}  L_{21}^T + L_{22} L_{22}^T \\
$$

Reescribiéndola, nos queda:

$$
A_{22} - L_{21}  L_{21}^T = L_{22} L_{22}^T \\
$$

Y aquí, un pequeño salto de fe (esperemos razonable para los que ya hayan cursado Algo 2): si resolvemos la factorización de Cholesky de $A_{22} - L_{21} L_{12}^T$, podemos obtener los dos bloques que nos faltan.





**Ejercicio**:  Calcular Cholesky en bloques para que tome una matriz $A \in \mathbb{R}^{n \times n}$ y nos devuelva $L$

La resolución tiene que ser *iterativa* pero puede convenir empezar a pensarlo en términos recursivos.

¿Cómo aseguran paso a paso que la matriz tenga factorización Cholesky?

**Hint**: Ojo con la dimensión de los vectores. Si queremos multiplicar una matriz "columna", con una matriz "fila" tenemos que hacer reshape primero o usar `np.outer`


In [6]:
def chol_en_bloques(A):
    """
    Dada una matriz A, devuelve L 
    
    Argumentos:
    -----------
    
    A: np.array
        Matriz de floats

    Devuelve:
    ---------
    
    L : np.array
        Descomposición Cholesky de A
        
    """
    
    """
    current_matrix es la matriz a la cual le estoy calculando la factorización de Cholesky
    
    Primero va a ser A, luego A22 - L21 * L21^T, ...
    """
    current_matrix = A.copy()
    
    """
    Vamos a ir "rellenando" paso a paso la factorización
    """
    L = np.zeros(A.shape)
    
    n = A.shape[0]
    
    """
    Vamos a iterar desde 0 hasta n-1 e ir completando 
    de acuerdo a las ecuaciones antes explicadas
    """
    for i in range(n):
        """
        TODO: Rellenar los valores de L[i, i]
        
        
        Observación: 
        
        En cada paso i estamos "llenando" las columnas y filas i-ésimas de L. Por eso tenemos que indexar por i
        
        Sin embargo, current_matrix la tenemos que indexar en 0 ya que es la matriz que vamos a ir
        "achicando" en dimensión. 
        
        """
        
        """
        TODO: ACA CHEQUEAR SI ES DP
        Cambiar el None por una condición
        
        """
        if None:
            raise ValueError("Matriz no SDP")
            
        L[i, i] = None
        """
        Caso "base": si es el final, no seguir
        """
        if i == n-1:
            break
        
        """
        TODO: Calcular los nuevos valores de L
        """
        L[i+1:, i] = None
        
        """
        TODO: Calcular la matriz del caso "recursivo".
        
        Esto sería la nueva "A"
        
        Sugerencia: usar np.outer o hacer reshape
        """
        current_dim = current_matrix.shape[0]
        
        L21 = L[i+1:, i]
        
        new_matrix = None

        """
        Asignamos la nueva matriz a calcular LU
        
        Nos aseguramos de que su dimensión se haya reducido en uno
        """
        current_matrix = new_matrix
        
        assert(current_matrix.shape == (current_dim-1, current_dim-1))

    return L


In [ ]:
tests = []

@mntest
def matriz_no_simetrica():
    A = np.array([
          [0.066076,.181880],
          [1.181880,.624953]
    ])

    try:
        L = chol_en_bloques(A)
        raise AssertionError("Tiene que lanzar excepción")
    except ValueError as e:
        pass


@mntest
def matriz_no_sdp():
    A = np.array([
          [-0.066076,   0.181880],
          [ 0.181880,   0.624953]
    ])
    try:
        L = chol_en_bloques(A)
        raise AssertionError("Tiene que lanzar excepción")
    except ValueError as e:
        pass

    
@mntest
def matriz_diagonal_positiva():
    A = np.array([
          [4.0,.0],
          [.0,9.0]
    ])
    L_1 = np.array([
          [2.0,0.0],
          [.0,3.0]
    ])
    
    L2 = chol_en_bloques(A)
    
    assert(np.allclose(L2,L_1))
    
    
@mntest
def matriz_sdp1():
    A  = np.array([[4.,-1.,1.], [-1, 4.25, 2.75],[1,2.75,3.5]],dtype='f')
    L = chol_en_bloques(A)
    
    assert(np.allclose(A,L@L.T))
    
    
@mntest
def matriz_sdp2():
       
    L_1 =  np.array([
        [5.99246,   .0,  .0,   .0,   .0],
        [0.45174,   5.02065,   .0,   .0,   .0],
        [0.50921,   0.40611,   5.25767,   .0,   .0],
        [0.45632,   0.52172,   0.78430,   5.98219,   .0],
        [0.41578,   0.11483,   0.24938,   0.45188,   5.28701]
    ])
    
    A    = L_1@L_1.T
    L2   = chol_en_bloques(A)
    assert(np.allclose(L2,L_1))

    
correr_tests()
